In [15]:
;from setup.txt

single_path = './'
; net_params
dt = 0.001  ; in ms
inv_dt = long(1000)
update = inv_dt

N_e  = long(80*10)
N_ir = long(15*10)
N_if = long(5*10)
N_all = N_e + N_ir + N_if


t_init=500*inv_dt  &  t_pre=0*inv_dt  &  t_stim=500*inv_dt  &  t_post=100*inv_dt
t_all = t_init + t_pre + t_stim + t_post

In [16]:
;from c_sims.txt
rseed = 'v01'
;rseed = 'v03'
con_t = 0.
con_r = 0.
con_p = 0.

In [17]:
str = 'test'

In [18]:
; Get seed for random number generator
if keyword_set(rseed) then begin
  if (size(rseed,/type) EQ 7) then begin  seed=rseed
    seed = lonarr(36)
;    openr, fu, single_path + rseed + '_rand.dat', /get_lun
;    readu, fu, seed  &  free_lun, fu
  endif  else  seed=long(rseed)
endif else  dum=randomu(seed,1)

print, seed

           0           0           0           0           0           0
           0           0           0           0           0           0
           0           0           0           0           0           0
           0           0           0           0           0           0
           0           0           0           0           0           0
           0           0           0           0           0           0


In [31]:
if not(keyword_set(ampa_scl)) then  ampa_scl=1. $
else  print, 'ctx: ampa=', ampa_scl

if not(keyword_set(nmda_e)) then  nmda_e=0. $
else  print, 'ctx: nmda_e=', nmda_e
if not(keyword_set(nmda_rs)) then  nmda_rs=0. $
else  print, 'ctx: nmda_rs=', nmda_rs
if not(keyword_set(nmda_fs)) then  nmda_fs=0. $
else  print, 'ctx: nmda_fs=', nmda_fs
nmda_scl = $
[fltarr(N_e)+(0.45*(1. - nmda_e)), fltarr(N_ir)+(0.1*(1. - nmda_rs)), fltarr(N_if)+(0.1*(1. - nmda_fs))]

gaba=0
con_t=0
con_p=0
con_r=0
con_if=0
con_ir=0
con_i_all=0
i_del=0
noise_wt=0
W_scl=0
decay_gaba=5.

In [32]:
W = fltarr(N_all,N_all)
W_noise = fltarr(N_all)

In [33]:
if not(keyword_set(noise_wt)) then  noise_wt=0.016
print, noise_wt

    0.0160000


In [34]:
if not(keyword_set(gaba)) then  gaba=0.
print, 'init_net: gaba = ', gaba

init_net: gaba =       0.00000


In [35]:
; Set up W by determining units to which unit i makes connections
; E->E
i=0 & e_e_con=fix(N_e*.10)
while (i LE N_e-1) do begin
ix = fix(randomu(seed,e_e_con) * N_e)
if (n_elements(uniq(ix,sort(ix))) EQ e_e_con) then begin
  W[ix,i] = fltarr(e_e_con) + 1.
  i+=1
endif
endwhile
print, 'e-e done'

; E->Ir
i=0 & e_ir_con=N_ir*.40
while (i LE N_e-1) do begin
a=randomu(seed,N_ir)  &  ix=uniq(a,sort(a))
if (n_elements(ix) EQ N_ir) then begin
  W[N_e+ix[0:e_ir_con-1],i] = fltarr(e_ir_con) + 1.
  i+=1
endif
endwhile
print, 'e-ir done'

; E->If
i=0 & e_if_con=N_if*.40
while (i LE N_e-1) do begin
ix = fix(randomu(seed,e_if_con) * N_if)
if (n_elements(uniq(ix,sort(ix))) EQ e_if_con) then begin
  W[ix+N_e+N_ir,i] = fltarr(e_if_con) + 1.
  i+=1
endif
endwhile
print, 'e-if done'

; Ir->E
i=N_e & ir_e_con=N_e*.50
while (i LE N_e+N_ir-1) do begin
a=randomu(seed,N_e)  &  ix=uniq(a,sort(a))
if (n_elements(ix) EQ N_e) then begin
  W[ix[0:ir_e_con-1],i] = fltarr(ir_e_con) - 1.
  i+=1
endif
endwhile
print, 'ir-e done'

; Ir->Ir
i=N_e & ir_ir_con=N_ir*.15
while (i LE N_e+N_ir-1) do begin
a=randomu(seed,N_ir)  &  ix=uniq(a,sort(a))
if (n_elements(ix) EQ N_ir) then begin
  W[N_e+ix[0:ir_ir_con-1],i] = fltarr(ir_ir_con) - 1.
  i+=1
endif
endwhile
print, 'ir-ir done'

; Ir->If
i=N_e & ir_if_con=N_if*.50
while (i LE N_e+N_ir-1) do begin
a = fix(randomu(seed,N_if) * 20000)
ix = uniq(a,sort(a))

if (n_elements(ix) EQ N_if) then begin
  W[ix[0:ir_if_con-1]+N_e+N_ir,i] = fltarr(ir_if_con) - 1.
  i+=1
endif
endwhile
print, 'ir-if done'

; If->E
i=N_e+N_ir & if_e_con=N_e*.50
while (i LE N_all-1) do begin
a=randomu(seed,N_e)  &  ix=uniq(a,sort(a))
if (n_elements(ix) EQ N_e) then begin
  W[ix[0:if_e_con-1],i] = fltarr(if_e_con) - 1.
  i+=1
endif
endwhile
print, 'if-e done'

; If->Ir
i=N_e+N_ir & if_ir_con=N_ir*.35
while (i LE N_all-1) do begin
a=randomu(seed,N_ir)  &  ix=uniq(a,sort(a))
if (n_elements(ix) EQ N_ir) then begin
  W[N_e+ix[0:if_ir_con-1],i] = fltarr(if_ir_con) - 1.
  i+=1
endif
endwhile
print, 'if-ir done'

; If->If
i=N_e+N_ir & if_if_con=N_if*.60
while (i LE N_all-1) do begin
a=randomu(seed,N_if)  &  ix=uniq(a,sort(a))
if (n_elements(ix) EQ N_if) then begin
  W[N_e+N_ir+ix[0:if_if_con-1],i] = fltarr(if_if_con) - 1.
  i+=1
endif
endwhile
print, 'if-if done'

W[0:N_e-1, 0:N_e-1] *= 0.01         ; E->E
W[N_e:N_e+N_ir-1, 0:N_e-1] *= 0.008  ; E->Ir
W[N_e+N_ir:*, 0:N_e-1] *= 0.019      ; E->If

W[0:N_e-1, N_e:N_e+N_ir-1] *= 0.008         ; Ir->E
W[N_e:N_e+N_ir-1, N_e:N_e+N_ir-1] *= 0.008  ; Ir->Ir
W[N_e+N_ir:*, N_e:N_e+N_ir-1] *= 0.008      ; Ir->If

W[0:N_e-1, N_e+N_ir:*] *= 0.019*(1. - gaba)        ; If->E
W[N_e:N_e+N_ir-1, N_e+N_ir:*] *= 0.01*(1. - gaba)  ; If->Ir
W[N_e+N_ir:*, N_e+N_ir:*] *= 0.01*(1. - gaba)      ; If->If

if keyword_set(W_scl) then begin
W *= W_scl
print, 'global weight scaling = ', W_scl
endif else $
W *= 0.0825  ; global weight scaling

W_noise = fltarr(N_all) + noise_wt   ; external noise input weights

e-e done
e-ir done
e-if done
ir-e done
ir-ir done
ir-if done
if-e done
if-ir done
if-if done


In [36]:
print, 'Initial # weights: ', total(W NE 0.)
print, '        # excitatory weights: ', total(W GT 0.)
print, '        # inhibitory weights: ', total(W LT 0.)
print, '        # E->E weights: ', total(W[0:N_e-1,0:N_e-1] GT 0.)

Initial # weights:       219150.
        # excitatory weights:       128000.
        # inhibitory weights:       91150.0
        # E->E weights:       64000.0


In [37]:
openw, fu, single_path + str + '_param.txt', /get_lun
printf, fu, N_all, N_e, N_ir, N_if, dt, inv_dt, update, t_init, t_pre, t_stim, t_post, t_all
free_lun, fu
openw, fu, single_path + str + '_weights.dat', /get_lun
writeu, fu, W, W_noise
free_lun, fu

In [38]:
;noise_gen.pro
freq = 100.  ; in Hz
noise = intarr(N_all,t_all/inv_dt)

for i=0,N_all-1 do begin
  a = randomu(seed, t_all/inv_dt, gamma=1.)/freq
  spike_times = total(long(1000.*a),/cumulative)
  ix = where(spike_times LT t_all/inv_dt, count)
  noise[i,spike_times[ix]] = 1
endfor

openw, fu, single_path + str + '_noise.dat', /get_lun
writeu, fu, noise  &  free_lun, fu

In [39]:
spikes = fltarr(N_all,inv_dt)
current_spike = inv_dt-1
v = dblarr(N_all)
temp = intarr(N_all)
noise = fltarr(N_all,update)

g_r_ampa=fltarr(N_all)  &  g_r_nmda=fltarr(N_all)  &  g_r_gaba=fltarr(N_all)
g_d_ampa=fltarr(N_all)  &  g_d_nmda=fltarr(N_all)  &  g_d_gaba=fltarr(N_all)
g_ampa=0.               &  g_nmda=0.               &  g_gaba=0.
inv_tau_r_ampa=-dt/.5   &  inv_tau_r_nmda=-dt/2.   &  inv_tau_r_gaba=-dt/.5
;inv_tau_d_ampa=-dt/2.   &  inv_tau_d_nmda=-dt/100. &  inv_tau_d_gaba=-dt/5.
inv_tau_d_ampa=-dt/2.   &  inv_tau_d_nmda=-dt/100. &  inv_tau_d_gaba=-dt/decay_gaba
norm_ampa=.472689       &  norm_nmda=.904821       &  norm_gaba=.697016

inv_tau_mem=-1./[fltarr(N_e)+20.,fltarr(N_ir)+20.,fltarr(N_if)+10.]  ; used in non-Izh version
;thresh = [fltarr(N_e)-52.,fltarr(N_ir)-55.,fltarr(N_if)-52.]   ; non-Izh
thresh = [fltarr(N_e)-52.,fltarr(N_ir)-52.,fltarr(N_if)-52.]   ; non-Izh
;thresh = [fltarr(N_e)+30.,fltarr(N_ir)+30.,fltarr(N_if)+30.]   ; Izh


In [40]:
openw, spike_fu, single_path + str + '_spk.dat', /get_lun
openw, Vm_fu, single_path + str + '_Vm.dat', /get_lun
openr, noise_fu, single_path + str + '_noise.dat', /get_lun

;spikes[*,*]=0  &  v[*]=double(-65.)  &  g_ampa[*]=0.  &  g_nmda[*]=0.  &  g_gaba[*]=0.  ; Izh
spikes[*,*]=0  &  v[*]=double(-70.)  &  g_ampa[*]=0.  &  g_nmda[*]=0.  &  g_gaba[*]=0.  ; non-Izh


In [41]:

for t=long(0),t_all-1 do begin
  fired = where(v GE thresh, count)
  if (count GT 0) then begin
    v[fired] = double(-59.)   ; non-Izh
    ;v[fired] = c[fired]  ; Izh
    ;u[fired] += d[fired]  ; Izh
    spikes[fired,current_spike] = 1.
  endif

  if ((t mod update) EQ 0) then begin
    readu, noise_fu, temp
    noise[*,0] = temp
    writeu, spike_fu, fix(total(spikes[*,*],2) GT 0)
    writeu, Vm_fu, v
  endif

  glu_act = (W > 0.)#spikes[*,0] + W_noise*noise[*,t mod update]

  g_r_ampa += inv_tau_r_ampa*g_r_ampa + glu_act
  g_d_ampa += inv_tau_d_ampa*g_d_ampa + glu_act
  g_ampa = (g_d_ampa - g_r_ampa)/norm_ampa

  g_r_nmda += inv_tau_r_nmda*g_r_nmda + glu_act
  g_d_nmda += inv_tau_d_nmda*g_d_nmda + glu_act
  g_nmda = (g_d_nmda - g_r_nmda)/norm_nmda

  g_r_gaba += inv_tau_r_gaba*g_r_gaba - (W < 0.)#spikes[*,0]
  g_d_gaba += inv_tau_d_gaba*g_d_gaba - (W < 0.)#spikes[*,0]
  g_gaba = (g_d_gaba - g_r_gaba)/norm_gaba

  I_syn = -(ampa_scl*g_ampa*v + nmda_scl*g_nmda*v/(1. + exp(-0.062*v)/3.57) + g_gaba*(v+70.))

  v += dt*(inv_tau_mem*(v + 70.) + I_syn)   ; non-Izh
  ;v += dt*(0.04 * v^2 + 5.*v + 140. - u + I_syn)
  ;u += dt*(a * ((b * v) - u))

  spikes=shift(spikes,0,-1)  &  spikes[*,current_spike]=0.
end ; t


In [42]:
free_lun, spike_fu, Vm_fu, noise_fu

if (total(finite(v)) NE N_all) then  print, 'error: v=NaN'
print, 'ctx: ', str, ' done'
return
end

% Lexer/Parser syntax error: expecting EOF, found 'A_ADFADFW'
  At: /home/yi/tmp2k3n9eyw, Line 7  Column 1
